Cleaning new data; identify which of the duplicated LANs are valid

In [1]:
import pandas as pd
import numpy as np

In [2]:
new_data = pd.read_csv(r"C:\Users\emanlangit\Documents\2025\00 ACSIN\New Data\SCM_Data_060125_input.csv")
new_data.rename(columns={'LOAN_ACCOUNT_NO': 'loan_account_no'}, inplace=True)

In [3]:
valid_LAN_buckets = pd.read_csv(r"C:\Users\emanlangit\Documents\2025\00 ACSIN\New Data\Data_Req_2023_2024_input.csv")

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_7872\2457353041.py:1: DtypeWarning: Columns (14,15,16,17,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_LAN_buckets = pd.read_csv(r"C:\Users\emanlangit\Documents\2025\00 ACSIN\New Data\Data_Req_2023_2024_input.csv")


In [4]:
valid_LANs = valid_LAN_buckets.loan_account_no.unique().tolist()

In [5]:
new_data.loan_account_no.nunique()

127611

In [6]:
new_data_valid_LANs_wdup = new_data[new_data['loan_account_no'].isin(valid_LANs)]
new_data_valid_LANs_wdup.loan_account_no.nunique()

52179

In [7]:
new_data_valid_LANs_wdup.loan_account_no.shape

(56752,)

In [52]:
gu = new_data_valid_LANs_wdup.loan_account_no.value_counts() > 1

In [58]:
#new_data_valid_LANs_wdup.loan_account_no.tolist()

In [59]:
new_data_valid_LANs_nodup = new_data_valid_LANs_wdup[~new_data_valid_LANs_wdup.duplicated('loan_account_no', keep=False)]

In [63]:
new_data_valid_LANs_nodup2 = new_data_valid_LANs_wdup[~new_data_valid_LANs_wdup.duplicated('loan_account_no', keep='first')]

In [65]:
new_data_valid_LANs_nodup2.loan_account_no.value_counts().sum()

np.int64(52179)

In [62]:
#52179 - 47607 = 4572 nodup false


4572

In [66]:
3663/51000

0.0718235294117647

In [51]:
new_data_valid_LANs_without_dup = new_data_valid_LANs_wdup

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_7872\2800502152.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_data_valid_LANs_without_dup = new_data_valid_LANs_wdup[gu]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [8]:
new_data_valid_LANs_wdup[new_data_valid_LANs_wdup.loan_account_no == 'TW01010101000002231339']

,APPLICATION_NUMBER,CIFNUMBER,CUSTOMER_NUMBER,Application Received Date,Product,Judgement date,DECISION,USER_COMMENT,loan_account_no,LOAN_SANCTIONED_DATE,...,Branch Name,Occupation Type,Industry,Sanctioned ROI,Sanction Tenure,Total Asset Cost,CIBIL score,Unnamed: 29,Unnamed: 30,Unnamed: 31
96337,APPL00137070,GLBCUST00000119780,CUST00123240,03-01-23 04:26:52.188000000 PM,2 Wheeler Loan,03-01-23 05:23:49.000000000 PM,Approve,no test code,TW01010101000002231339,03-01-23,...,Eastern Mumbai,Self Employed Non Professional,Others,11.29,36,110050,734.0,NaN,NaN,NaN
156798,APPL00137070,GLBCUST00000119780,CUST00123240,03-01-23 04:26:52.188000000 PM,2 Wheeler Loan,03-01-23 05:23:49.000000000 PM,Approve,no test code,TW01010101000002231339,03-01-23,...,Eastern Mumbai,Self Employed Non Professional,Others,11.29,36,110050,734.0,NaN,NaN,NaN


In [12]:
#dealing with duplicated LANs
cleaning_df = new_data_valid_LANs_wdup.copy()
# Identify duplicated values
duplicates = cleaning_df.duplicated('loan_account_no', keep='first')

# Create a counter to keep track of duplicates
counter = {}

# Loop through the DataFrame and rename duplicates
for idx, value in cleaning_df.loc[duplicates, 'loan_account_no'].items():
    if value not in counter:
        counter[value] = 1
    counter[value] += 1
    cleaning_df.at[idx, 'loan_account_no'] = f"{value}_duplicate{counter[value] - 1}"

In [16]:
#cleaning_df.loan_account_no.unique().tolist()

In [10]:
# duplicate_counts = cleaning_df['loan_account_no'].value_counts()
# duplicates = duplicate_counts[duplicate_counts > 1]

In [17]:
cleaning_df.loan_account_no.value_counts() #df with valid LANs and duplicated LANs

loan_account_no
TW01020102000002281768               1
TW01010102000002277004               1
TW01010101000002275484_duplicate1    1
TW01030107000002275051               1
TW01030107000002271602               1
                                    ..
TW01020101000002232620               1
TW01010105000002233875               1
TW01020102000002231359               1
TW01030105000002235593               1
TW01020101000002232829               1
Name: count, Length: 56752, dtype: int64

In [18]:
#load base data
base_data_old = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\01 ACSIN EDA\Data\BASE DATA FOR EDA_1.csv")

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_7872\941017695.py:2: DtypeWarning: Columns (1,2,3,4,5,6,166,170) have mixed types. Specify dtype option on import or set low_memory=False.
  base_data_old = pd.read_csv(r"C:\Users\emanlangit\Documents\ACSIN\01 ACSIN EDA\Data\BASE DATA FOR EDA_1.csv")


In [19]:
reduction_cols = ['loan_account_no', 'res_years_at_current_city', 'res_resident_status','res_city',
                 'res_state','total_income','occupation_type', 'industry','bureau_score']
reduced_base_data_old = base_data_old[reduction_cols]

In [20]:
scm_LAN = set(new_data.loan_account_no.unique().tolist())
valid_LAN = set(valid_LAN_buckets.loan_account_no.unique().tolist())

diff = valid_LAN - scm_LAN
missing_LANs = list(diff)

In [21]:
valid_reduced_base_data_old = reduced_base_data_old[reduced_base_data_old['loan_account_no'].isin(valid_LANs)]
#valid_reduced_base_data_old.loan_account_no.nunique()
adjusted_valid_reduced_base_data_old = valid_reduced_base_data_old[~valid_reduced_base_data_old['loan_account_no'].isin(missing_LANs)]
adjusted_valid_reduced_base_data_old.loan_account_no.nunique()

52179

In [22]:
cleaning_df.loan_account_no.nunique() #duped rows 56752; 56752 - 52179 = 4573

56752

In [23]:
#merge valid LANs and new data
merged_new_base = cleaning_df.merge(adjusted_valid_reduced_base_data_old,how='outer',on=['loan_account_no'])

In [24]:
merged_new_base.shape

(56752, 40)

In [25]:
new_data_cols = ['Years at Current City', 'Resident Status', 'City', 'State','Total Income',
                'Occupation Type', 'Industry', 'CIBIL score']
compare_cols = new_data_cols + reduction_cols

In [26]:
reduced_merged_new_base = merged_new_base[compare_cols].copy()

In [27]:
reduced_merged_new_base.columns

Index(['Years at Current City', 'Resident Status', 'City', 'State',
       'Total Income', 'Occupation Type', 'Industry', 'CIBIL score',
       'loan_account_no', 'res_years_at_current_city', 'res_resident_status',
       'res_city', 'res_state', 'total_income', 'occupation_type', 'industry',
       'bureau_score'],
      dtype='object')

In [40]:
reduced_merged_new_base.shape

(56752, 25)

In [30]:
reduced_merged_new_base['years_at_city_check'] = reduced_merged_new_base['Years at Current City'] == reduced_merged_new_base['res_years_at_current_city']
reduced_merged_new_base['resident_status_check'] = reduced_merged_new_base['Resident Status'] == reduced_merged_new_base['res_resident_status']
reduced_merged_new_base['city_check'] = reduced_merged_new_base['City'] == reduced_merged_new_base['res_city']
reduced_merged_new_base['state_check'] = reduced_merged_new_base['State'] == reduced_merged_new_base['res_state']
reduced_merged_new_base['income_check'] = reduced_merged_new_base['Total Income'] == reduced_merged_new_base['total_income']
reduced_merged_new_base['occupation_check'] = reduced_merged_new_base['Occupation Type'] == reduced_merged_new_base['occupation_type']
reduced_merged_new_base['industry_check'] = reduced_merged_new_base['Industry'] == reduced_merged_new_base['industry']
reduced_merged_new_base['CIBIL_check'] = reduced_merged_new_base['CIBIL score'] == reduced_merged_new_base['bureau_score']

In [48]:
do = reduced_merged_new_base[~reduced_merged_new_base['years_at_city_check']][['loan_account_no','Years at Current City','res_years_at_current_city' ]]

In [51]:
re = reduced_merged_new_base[reduced_merged_new_base['years_at_city_check']][['loan_account_no','Years at Current City','res_years_at_current_city' ]]

In [52]:
re.loan_account_no.nunique()

50332

In [32]:
diff_rows = reduced_merged_new_base[reduced_merged_new_base.eq(False).any(axis=1)]

In [33]:
diff_rows

,Years at Current City,Resident Status,City,State,Total Income,Occupation Type,Industry,CIBIL score,loan_account_no,res_years_at_current_city,...,industry,bureau_score,years_at_city_check,resident_status_check,city_check,state_check,income_check,occupation_check,industry_check,CIBIL_check
9,0,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,560000.0,Self Employed Non Professional,Apparels,757.0,TW01010101000002231217,0.0,...,Apparels,757.0,True,True,True,True,True,True,True,True
12,0,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,390000.0,Self Employed Non Professional,Others,739.0,TW01010101000002231242,22.0,...,Others,739.0,False,True,True,True,True,True,True,True
13,22,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,390000.0,Self Employed Non Professional,Others,739.0,TW01010101000002231242_duplicate1,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
15,0,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,390000.0,Self Employed Non Professional,Others,734.0,TW01010101000002231339,22.0,...,Others,734.0,False,True,True,True,True,True,True,True
16,22,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,390000.0,Self Employed Non Professional,Others,734.0,TW01010101000002231339_duplicate1,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56736,5,Rented,GHAZIABAD,UTTAR PRADESH,420000.0,Self Employed Non Professional,Others,-1.0,UT01030107000002270269_duplicate1,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
56739,25,Owned by Parent /Sibling,GAUTAM BUDDHA NAGAR,UTTAR PRADESH,420000.0,Self Employed Non Professional,Others,768.0,UT01030107000002272618,5.0,...,Others,768.0,False,False,False,True,True,True,True,True
56740,5,Rented,GHAZIABAD,UTTAR PRADESH,420000.0,Self Employed Non Professional,Others,768.0,UT01030107000002272618_duplicate1,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
56745,10,Owned by Parent /Sibling,GHAZIABAD,UTTAR PRADESH,430000.0,Self Employed Non Professional,Others,-1.0,UT01030107000002280635,16.0,...,Others,-1.0,False,False,True,True,True,True,True,True


In [53]:
reduced_merged_new_base['resident_status_check'] = reduced_merged_new_base['Resident Status'] == reduced_merged_new_base['res_resident_status']
reduced_merged_new_base['resident_status_check'].value_counts()

C:\Users\emanlangit.AEONCREDIT\AppData\Local\Temp\ipykernel_11868\4214886488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_merged_new_base['resident_status_check'] = reduced_merged_new_base['Resident Status'] == reduced_merged_new_base['res_resident_status']


resident_status_check
True     51156
False     5596
Name: count, dtype: int64

In [56]:
test = reduced_merged_new_base['resident_status_check']==reduced_merged_new_base['years_at_city_check']

In [58]:
test.value_counts()

True     54938
False     1814
Name: count, dtype: int64

In [34]:
ok_rows = reduced_merged_new_base[~reduced_merged_new_base.eq(False).any(axis=1)]

In [35]:
ok_rows

,Years at Current City,Resident Status,City,State,Total Income,Occupation Type,Industry,CIBIL score,loan_account_no,res_years_at_current_city,...,industry,bureau_score,years_at_city_check,resident_status_check,city_check,state_check,income_check,occupation_check,industry_check,CIBIL_check
0,15,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,420000.0,Salaried,Others,763.0,TW01010101000002230893,15.0,...,Others,763.0,True,True,True,True,True,True,True,True
1,35,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,420000.0,Salaried,Others,727.0,TW01010101000002230943,35.0,...,Others,727.0,True,True,True,True,True,True,True,True
2,15,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,390000.0,Self Employed Non Professional,Others,747.0,TW01010101000002231047,15.0,...,Others,747.0,True,True,True,True,True,True,True,True
3,15,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,420000.0,Salaried,Others,780.0,TW01010101000002231074,15.0,...,Others,780.0,True,True,True,True,True,True,True,True
4,15,Owned by Parent /Sibling,MUMBAI,MAHARASHTRA,390000.0,Self Employed Non Professional,Others,740.0,TW01010101000002231096,15.0,...,Others,740.0,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56747,28,Owned by Parent /Sibling,GHAZIABAD,UTTAR PRADESH,420000.0,Self Employed Non Professional,Others,726.0,UT01030107000002280644,28.0,...,Others,726.0,True,True,True,True,True,True,True,True
56748,27,Owned by Parent /Sibling,GHAZIABAD,UTTAR PRADESH,430000.0,Self Employed Non Professional,Others,746.0,UT01030107000002282167,27.0,...,Others,746.0,True,True,True,True,True,True,True,True
56749,24,Rented,GHAZIABAD,UTTAR PRADESH,420000.0,Self Employed Non Professional,Others,-1.0,UT01030107000002282455,24.0,...,Others,-1.0,True,True,True,True,True,True,True,True
56750,15,Rented,GHAZIABAD,UTTAR PRADESH,420000.0,Self Employed Non Professional,Others,-1.0,UT01030107000002286165,15.0,...,Others,-1.0,True,True,True,True,True,True,True,True


In [41]:
reduced_merged_new_base.to_csv('merged_new_base_data.csv',index = False)